# Word Embeddings

## Loading Datasets & Importing Modules

In [5]:
import sys
import os
sys.path.append('./Files')
sys.path.append('./Datasets')

In [6]:
os.system('pip install nltk')

0

In [7]:
os.system('pip install gensim')

0

In [8]:
import nltk
import gensim
from gensim.models import Word2Vec
import pandas as pd
from dataCleaner import preProcess
from Sequencer import Sequencer
from helper_fns import write_dict
from helper_fns import read_dict
import numpy as np
import matplotlib.pyplot as plt
import json
import string

In [9]:
import warnings
warnings.filterwarnings(action = 'ignore')

In [10]:
df_train = pd.read_csv('Datasets/merged_train_datasets.csv')
df_train = df_train.rename(columns={"text":"tweet"})

try:
    df_train.drop(columns=['Unnamed: 0'], inplace = True)
except:
    pass

## Word2Vec

### Preparing Data

In [11]:
preProcess(df_train)
df_train.head()

,tweet,sentiment
0,اهنئ احمد جمال دين قيادي بحزب مصر بمناسب صدر ر...,positive
1,برادعي يستقو بامريكا مرهاخر يرسل عصام عريان اش...,negative
2,والده اقول بخاطر حشيش تضحك بس اقول كيلك تعطين ...,neutral
3,انتخبوا العرص انتخبوا البرص مرسي رئيس اين رئيس...,neutral
4,يتقال ستريكر صريح كاريوك السكه شمال,positive


In [12]:
tweets = []
for i in df_train['tweet']:
    tweet = nltk.word_tokenize(i)
    tweets.append(tweet)

### CBOW

In [13]:
tweets_size = len(tweets)
tweets_size

6637

In [14]:
flat_tweets = [item for sublist in tweets for item in sublist]
vocab = set(flat_tweets)

In [15]:
vocab_size = len(vocab)
embed_dim = 10
context_size = 2
vocab_size

18283

In [16]:
word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for i, word in enumerate(vocab)}

In [17]:
data = []
for i in range(tweets_size):
    for j in range(2, len(tweets[i]) - 2):
        context = [tweets[i][j - 2], tweets[i][j - 1], tweets[i][j + 1], tweets[i][j + 2]]
        target = tweets[i][j]
        data.append((context, target))
print(data[:7])
len(data)

[(['اهنئ', 'احمد', 'دين', 'قيادي'], 'جمال'), (['احمد', 'جمال', 'قيادي', 'بحزب'], 'دين'), (['جمال', 'دين', 'بحزب', 'مصر'], 'قيادي'), (['دين', 'قيادي', 'مصر', 'بمناسب'], 'بحزب'), (['قيادي', 'بحزب', 'بمناسب', 'صدر'], 'مصر'), (['بحزب', 'مصر', 'صدر', 'روايت'], 'بمناسب'), (['برادعي', 'يستقو', 'مرهاخر', 'يرسل'], 'بامريكا')]


35262

In [18]:
embeddings =  np.random.random_sample((vocab_size, embed_dim))
print(embeddings[:2])
len(embeddings)

[[0.14277518 0.81119964 0.00769278 0.3773396  0.13513933 0.82835643
  0.57586812 0.14529619 0.78881317 0.75376777]
 [0.78276127 0.17575534 0.47901921 0.83521343 0.27718658 0.23803615
  0.1102499  0.85755252 0.52170385 0.43743006]]


18283

In [19]:
def linear(m, theta):
    w = theta
    return m.dot(w)

def log_softmax(x):
    e_x = np.exp(x - np.max(x))
    return np.log(e_x / e_x.sum())

def NLLLoss(logs, targets):
    out = logs[range(len(targets)), targets]
    return -out.sum()/len(out)

def log_softmax_crossentropy_with_logits(logits,target):

    out = np.zeros_like(logits)
    out[np.arange(len(logits)),target] = 1
    
    softmax = np.exp(logits) / np.exp(logits).sum(axis=-1,keepdims=True)
    
    return (- out + softmax) / logits.shape[0]

def forward(context_idxs, theta):
    m = embeddings[context_idxs].reshape(1, -1)
    n = linear(m, theta)
    o = log_softmax(n)
    
    return m, n, o

def backward(preds, theta, target_idxs):
    m, n, o = preds
    
    dlog = log_softmax_crossentropy_with_logits(n, target_idxs)
    dw = m.T.dot(dlog)
    
    return dw

def optimize(theta, grad, lr=0.03):
    theta -= grad * lr
    return theta

In [20]:
theta = np.random.uniform(-1, 1, (2 * context_size * embed_dim, vocab_size))

In [21]:
epoch_losses = {}

for epoch in range(30):

    losses =  []

    for context, target in data:
        context_idxs = np.array([word_to_ix[w] for w in context])
        preds = forward(context_idxs, theta)

        target_idxs = np.array([word_to_ix[target]])
        loss = NLLLoss(preds[-1], target_idxs)

        losses.append(loss)

        grad = backward(preds, theta, target_idxs)
        theta = optimize(theta, grad, lr=0.03)
        
    epoch_losses[epoch] = losses

In [23]:
np.savetxt("Word Embeddings/embeddings.txt", embeddings)

In [30]:
CBoW = Word2Vec(tweets,
                vector_size=50,
                window=2,
                min_count=2)

In [31]:
CBoW_dict = {}
for word in CBoW.wv.key_to_index:
    CBoW_dict[word] = CBoW.wv[word].tolist()

In [32]:
len(CBoW.wv.key_to_index)

7020

In [33]:
CBoW.save("Word Embeddings/cbow_50_2_2.model")

In [34]:
write_dict("Word Embeddings/cbow-dict_50_2_2.txt", CBoW_dict)

### Skip-Gram

In [35]:
SkipGram = Word2Vec(tweets,
                    min_count=3,
                    window=2,
                    vector_size=50,
                    sorted_vocab=1,
                    sg=1)

In [36]:
embeddings_dict = {}
for word in SkipGram.wv.key_to_index:
    embeddings_dict[word] = SkipGram.wv[word].tolist()

In [37]:
# freq = {}
# for i, word in enumerate(nltk.tokenize.wordpunct_tokenize(" ".join(df_train['tweet'].to_numpy().flatten()))):
#     try:
#         freq[word] += 1
#     except:
#         freq[word] = 1
# sorted_freq = {key: val for key, val in sorted(freq.items(), key = lambda ele: ele[1], reverse=1)}

In [38]:
len(SkipGram.wv.key_to_index)

4349

In [39]:
sequencer = Sequencer(df_train['tweet'], embeddings_dict, 1000)

In [40]:
df_train.iloc[0,0]

'اهنئ احمد جمال دين قيادي بحزب مصر بمناسب صدر روايت'

In [41]:
(sequencer.text_to_vec(df_train.iloc[0,0])[0:50] == SkipGram.wv["اهنئ"]).sum()

50

In [42]:
(sequencer.text_to_vec(df_train.iloc[0,0])).shape

(400,)

In [43]:
(sequencer.padder(sequencer.text_to_vec(df_train.iloc[0,0]))).shape

(1000,)

In [44]:
SkipGram.save("Word Embeddings/sg_3_2_50.model")

In [45]:
write_dict("Word Embeddings/dict_3_2_50.txt", embeddings_dict)

## GloVe